In [1]:
import json
import urllib
import pandas as pd
import numpy as np
import http
from urllib.error import URLError, HTTPError, ContentTooShortError
from datetime import datetime
import time

def download(url, num_retries=5): 
#     print('Downloading:', url)
    try: 
        time.sleep(1)
        html = urllib.request.urlopen(url).read()
    except (URLError, HTTPError, ContentTooShortError, http.client.HTTPException) as e: 
        print('Download error:', e.reason,url)
        html = None 
        if num_retries > 0: 
            if hasattr(e, 'code') and 500 <= e.code < 600: 
                time.sleep(10)
                # recursively retry 5xx HTTP errors 
                return download(url, num_retries - 1) 
    return html

def cbb_calendar(season):
    url = "http://site.api.espn.com/apis/site/v2/sports/basketball/mens-college-basketball/scoreboard?dates={}".format(season)
    resp = download(url=url)
    txt = json.loads(resp)['leagues'][0]['calendar']
    datenum = list(map(lambda x: x[:10].replace("-",""),txt))
    date = list(map(lambda x: x[:10],txt))
    
    year = list(map(lambda x: x[:4],txt))
    month = list(map(lambda x: x[5:7],txt))
    day = list(map(lambda x: x[8:10],txt))
    
    data = {"season": season,
            "datetime" : txt,
            "date" : date,
            "year": year,
            "month": month,
            "day": day,
            "dateURL": datenum
            
           }
    df = pd.DataFrame(data)
    df['url']="http://site.api.espn.com/apis/site/v2/sports/football/college-football/scoreboard?dates="
    df['url']= df['url'] + df['dateURL']
    return df

cbb_calendar(2018)

,season,datetime,date,year,month,day,dateURL,url
0,2018,2017-11-10T08:00Z,2017-11-10,2017,11,10,20171110,http://site.api.espn.com/apis/site/v2/sports/f...
1,2018,2017-11-11T08:00Z,2017-11-11,2017,11,11,20171111,http://site.api.espn.com/apis/site/v2/sports/f...
2,2018,2017-11-12T08:00Z,2017-11-12,2017,11,12,20171112,http://site.api.espn.com/apis/site/v2/sports/f...
3,2018,2017-11-13T08:00Z,2017-11-13,2017,11,13,20171113,http://site.api.espn.com/apis/site/v2/sports/f...
4,2018,2017-11-14T08:00Z,2017-11-14,2017,11,14,20171114,http://site.api.espn.com/apis/site/v2/sports/f...
...,...,...,...,...,...,...,...,...
137,2018,2018-03-28T07:00Z,2018-03-28,2018,03,28,20180328,http://site.api.espn.com/apis/site/v2/sports/f...
138,2018,2018-03-29T07:00Z,2018-03-29,2018,03,29,20180329,http://site.api.espn.com/apis/site/v2/sports/f...
139,2018,2018-03-30T07:00Z,2018-03-30,2018,03,30,20180330,http://site.api.espn.com/apis/site/v2/sports/f...
140,2018,2018-03-31T07:00Z,2018-03-31,2018,03,31,20180331,http://site.api.espn.com/apis/site/v2/sports/f...


In [ ]:
pd.set_option('display.max_colwidth',None)
schedule = {}
for x in range(2001,2022):
    full_schedule = cbb_calendar(x)
    full_schedule.to_json(orient='records')
    schedule[x]=full_schedule
    
data_dict = {
    key: schedule[key].to_dict(orient='records')
    for key in schedule.keys()
}

with open('cbb_schedule.json','w') as fp:
    json.dump(
    data_dict,
    fp,
    indent=4,
    sort_keys=True)    

In [2]:
import json
import urllib
import pandas as pd
import numpy as np
import http
from urllib.error import URLError, HTTPError, ContentTooShortError
from datetime import datetime
import time

def download(url, num_retries=5): 
#     print('Downloading:', url)
    try: 
        time.sleep(1)
        html = urllib.request.urlopen(url).read()
    except (URLError, HTTPError, ContentTooShortError, http.client.HTTPException) as e: 
        print('Download error:', e.reason,url)
        html = None 
        if num_retries > 0: 
            if hasattr(e, 'code') and 500 <= e.code < 600: 
                time.sleep(10)
                # recursively retry 5xx HTTP errors 
                return download(url, num_retries - 1) 
    return html

def cbb_schedule(year):
    url = "http://site.api.espn.com/apis/site/v2/sports/basketball/mens-college-basketball/scoreboard?dates={}".format(year)
    resp = download(url=url)
    txt = json.loads(resp)['leagues'][0]['calendar']
#     print(len(txt))
    txt = list(map(lambda x: x[:10].replace("-",""),txt))
    
    ev = pd.DataFrame()
    i=0
    for date in txt:
        print(f"Working on {year}: {i+1} of {len(txt)}, date: {txt[i]}")
        url = "http://site.api.espn.com/apis/site/v2/sports/basketball/mens-college-basketball/scoreboard?groups=50&dates={}&limit=1000".format(date)
        resp = download(url=url)
        events_txt = json.loads(resp)
        events = events_txt['events']
        for event in events:
            bad_keys = ['linescores', 'statistics', 'leaders',  'records']
            for k in bad_keys:
                if k in event['competitions'][0]['competitors'][0].keys():
                    del event['competitions'][0]['competitors'][0][k]
                if k in event['competitions'][0]['competitors'][1].keys():
                    del event['competitions'][0]['competitors'][1][k]
            if 'links' in event['competitions'][0]['competitors'][0]['team'].keys():
                del event['competitions'][0]['competitors'][0]['team']['links']
            if 'links' in event['competitions'][0]['competitors'][1]['team'].keys():
                del event['competitions'][0]['competitors'][1]['team']['links']    
            if event['competitions'][0]['competitors'][0]['homeAway']=='home':
                event['competitions'][0]['home'] = event['competitions'][0]['competitors'][0]['team']    
            else: 
                event['competitions'][0]['away'] = event['competitions'][0]['competitors'][0]['team']
            if event['competitions'][0]['competitors'][1]['homeAway']=='away':
                event['competitions'][0]['away'] = event['competitions'][0]['competitors'][1]['team']
            else: 
                event['competitions'][0]['home'] = event['competitions'][0]['competitors'][1]['team']

            del_keys = ['competitors', 'broadcasts','geoBroadcasts', 'headlines']
            for k in del_keys:
                if k in event['competitions'][0].keys():
                    del event['competitions'][0][k]

            ev = ev.append(pd.json_normalize(event['competitions'][0]))
        i+=1
    ev['season']=year
    return ev

schedule_table = pd.DataFrame()
schedule = {}
for x in range(2002,2022):
    year_schedule = cbb_schedule(x)
    year_schedule.to_csv(f"cbb/schedules/cbb_games_info_{x}.csv")
    schedule_table = schedule_table.append(year_schedule)
    year_schedule.to_json(orient='records')
    schedule[x]=year_schedule
    
data_dict = {
    key: schedule[key].to_dict(orient='records')
    for key in schedule.keys()
}

with open('cbb_schedule.json','w') as fp:
    json.dump(
    data_dict,
    fp,
    indent=4,
    sort_keys=True)

schedule_table.to_csv('cbb_games_info_2002_2021.csv')
    
# event['competitions'][0]

Working on 2002: 1 of 136, date: 20011108
Working on 2002: 2 of 136, date: 20011109
Working on 2002: 3 of 136, date: 20011112
Working on 2002: 4 of 136, date: 20011113
Working on 2002: 5 of 136, date: 20011114
Working on 2002: 6 of 136, date: 20011115
Working on 2002: 7 of 136, date: 20011116
Working on 2002: 8 of 136, date: 20011117
Working on 2002: 9 of 136, date: 20011118
Working on 2002: 10 of 136, date: 20011119
Working on 2002: 11 of 136, date: 20011120
Working on 2002: 12 of 136, date: 20011121
Working on 2002: 13 of 136, date: 20011122
Working on 2002: 14 of 136, date: 20011123
Working on 2002: 15 of 136, date: 20011124
Working on 2002: 16 of 136, date: 20011125
Working on 2002: 17 of 136, date: 20011126
Working on 2002: 18 of 136, date: 20011127
Working on 2002: 19 of 136, date: 20011128
Working on 2002: 20 of 136, date: 20011129
Working on 2002: 21 of 136, date: 20011130
Working on 2002: 22 of 136, date: 20011201
Working on 2002: 23 of 136, date: 20011202
Working on 2002: 24 

Working on 2003: 37 of 138, date: 20021220
Working on 2003: 38 of 138, date: 20021221
Working on 2003: 39 of 138, date: 20021222
Working on 2003: 40 of 138, date: 20021223
Working on 2003: 41 of 138, date: 20021227
Working on 2003: 42 of 138, date: 20021228
Working on 2003: 43 of 138, date: 20021229
Working on 2003: 44 of 138, date: 20021230
Working on 2003: 45 of 138, date: 20021231
Working on 2003: 46 of 138, date: 20030101
Working on 2003: 47 of 138, date: 20030102
Working on 2003: 48 of 138, date: 20030103
Working on 2003: 49 of 138, date: 20030104
Working on 2003: 50 of 138, date: 20030105
Working on 2003: 51 of 138, date: 20030106
Working on 2003: 52 of 138, date: 20030107
Working on 2003: 53 of 138, date: 20030108
Working on 2003: 54 of 138, date: 20030109
Working on 2003: 55 of 138, date: 20030110
Working on 2003: 56 of 138, date: 20030111
Working on 2003: 57 of 138, date: 20030112
Working on 2003: 58 of 138, date: 20030113
Working on 2003: 59 of 138, date: 20030114
Working on 

Working on 2004: 89 of 139, date: 20040211
Working on 2004: 90 of 139, date: 20040212
Working on 2004: 91 of 139, date: 20040213
Working on 2004: 92 of 139, date: 20040214
Working on 2004: 93 of 139, date: 20040215
Working on 2004: 94 of 139, date: 20040216
Working on 2004: 95 of 139, date: 20040217
Working on 2004: 96 of 139, date: 20040218
Working on 2004: 97 of 139, date: 20040219
Working on 2004: 98 of 139, date: 20040220
Working on 2004: 99 of 139, date: 20040221
Working on 2004: 100 of 139, date: 20040222
Working on 2004: 101 of 139, date: 20040223
Working on 2004: 102 of 139, date: 20040224
Working on 2004: 103 of 139, date: 20040225
Working on 2004: 104 of 139, date: 20040226
Working on 2004: 105 of 139, date: 20040227
Working on 2004: 106 of 139, date: 20040228
Download error: Gateway Time-out http://site.api.espn.com/apis/site/v2/sports/basketball/mens-college-basketball/scoreboard?groups=50&dates=20040228&limit=1000
Download error: Gateway Time-out http://site.api.espn.com/a

Working on 2005: 132 of 139, date: 20050324
Working on 2005: 133 of 139, date: 20050325
Working on 2005: 134 of 139, date: 20050326
Working on 2005: 135 of 139, date: 20050327
Working on 2005: 136 of 139, date: 20050329
Working on 2005: 137 of 139, date: 20050331
Working on 2005: 138 of 139, date: 20050402
Working on 2005: 139 of 139, date: 20050404
Working on 2006: 1 of 140, date: 20051108
Working on 2006: 2 of 140, date: 20051109
Working on 2006: 3 of 140, date: 20051110
Working on 2006: 4 of 140, date: 20051111
Working on 2006: 5 of 140, date: 20051112
Working on 2006: 6 of 140, date: 20051113
Working on 2006: 7 of 140, date: 20051114
Working on 2006: 8 of 140, date: 20051115
Working on 2006: 9 of 140, date: 20051116
Working on 2006: 10 of 140, date: 20051117
Working on 2006: 11 of 140, date: 20051118
Working on 2006: 12 of 140, date: 20051119
Working on 2006: 13 of 140, date: 20051120
Working on 2006: 14 of 140, date: 20051121
Working on 2006: 15 of 140, date: 20051122
Working on 2

Working on 2007: 32 of 140, date: 20061208
Working on 2007: 33 of 140, date: 20061209
Working on 2007: 34 of 140, date: 20061210
Working on 2007: 35 of 140, date: 20061211
Working on 2007: 36 of 140, date: 20061212
Working on 2007: 37 of 140, date: 20061213
Working on 2007: 38 of 140, date: 20061214
Working on 2007: 39 of 140, date: 20061215
Working on 2007: 40 of 140, date: 20061216
Working on 2007: 41 of 140, date: 20061217
Working on 2007: 42 of 140, date: 20061218
Working on 2007: 43 of 140, date: 20061219
Working on 2007: 44 of 140, date: 20061220
Working on 2007: 45 of 140, date: 20061221
Working on 2007: 46 of 140, date: 20061222
Working on 2007: 47 of 140, date: 20061223
Working on 2007: 48 of 140, date: 20061224
Working on 2007: 49 of 140, date: 20061227
Working on 2007: 50 of 140, date: 20061228
Working on 2007: 51 of 140, date: 20061229
Working on 2007: 52 of 140, date: 20061230
Working on 2007: 53 of 140, date: 20061231
Working on 2007: 54 of 140, date: 20070101
Working on 

...

Working on 2016: 20 of 142, date: 20151202
Working on 2016: 21 of 142, date: 20151203
Working on 2016: 22 of 142, date: 20151204
Working on 2016: 23 of 142, date: 20151205
Working on 2016: 24 of 142, date: 20151206
Working on 2016: 25 of 142, date: 20151207
Working on 2016: 26 of 142, date: 20151208
Working on 2016: 27 of 142, date: 20151209
Working on 2016: 28 of 142, date: 20151210
Working on 2016: 29 of 142, date: 20151211
Working on 2016: 30 of 142, date: 20151212
Working on 2016: 31 of 142, date: 20151213
Working on 2016: 32 of 142, date: 20151214
Working on 2016: 33 of 142, date: 20151215
Working on 2016: 34 of 142, date: 20151216
Working on 2016: 35 of 142, date: 20151217
Working on 2016: 36 of 142, date: 20151218
Working on 2016: 37 of 142, date: 20151219
Working on 2016: 38 of 142, date: 20151220
Working on 2016: 39 of 142, date: 20151221
Working on 2016: 40 of 142, date: 20151222
Working on 2016: 41 of 142, date: 20151223
Working on 2016: 42 of 142, date: 20151225
Working on 

Working on 2017: 57 of 143, date: 20170106
Working on 2017: 58 of 143, date: 20170107
Working on 2017: 59 of 143, date: 20170108
Working on 2017: 60 of 143, date: 20170109
Working on 2017: 61 of 143, date: 20170110
Working on 2017: 62 of 143, date: 20170111
Working on 2017: 63 of 143, date: 20170112
Working on 2017: 64 of 143, date: 20170113
Working on 2017: 65 of 143, date: 20170114
Working on 2017: 66 of 143, date: 20170115
Working on 2017: 67 of 143, date: 20170116
Working on 2017: 68 of 143, date: 20170117
Working on 2017: 69 of 143, date: 20170118
Working on 2017: 70 of 143, date: 20170119
Working on 2017: 71 of 143, date: 20170120
Working on 2017: 72 of 143, date: 20170121
Working on 2017: 73 of 143, date: 20170122
Working on 2017: 74 of 143, date: 20170123
Working on 2017: 75 of 143, date: 20170124
Working on 2017: 76 of 143, date: 20170125
Working on 2017: 77 of 143, date: 20170126
Working on 2017: 78 of 143, date: 20170127
Working on 2017: 79 of 143, date: 20170128
Working on 

Working on 2018: 74 of 142, date: 20180123
Working on 2018: 75 of 142, date: 20180124
Working on 2018: 76 of 142, date: 20180125
Working on 2018: 77 of 142, date: 20180126
Working on 2018: 78 of 142, date: 20180127
Working on 2018: 79 of 142, date: 20180128
Working on 2018: 80 of 142, date: 20180129
Working on 2018: 81 of 142, date: 20180130
Working on 2018: 82 of 142, date: 20180131
Working on 2018: 83 of 142, date: 20180201
Working on 2018: 84 of 142, date: 20180202
Working on 2018: 85 of 142, date: 20180203
Working on 2018: 86 of 142, date: 20180204
Working on 2018: 87 of 142, date: 20180205
Working on 2018: 88 of 142, date: 20180206
Working on 2018: 89 of 142, date: 20180207
Working on 2018: 90 of 142, date: 20180208
Working on 2018: 91 of 142, date: 20180209
Working on 2018: 92 of 142, date: 20180210
Working on 2018: 93 of 142, date: 20180211
Working on 2018: 94 of 142, date: 20180212
Working on 2018: 95 of 142, date: 20180213
Working on 2018: 96 of 142, date: 20180214
Working on 

Working on 2019: 122 of 152, date: 20190308
Working on 2019: 123 of 152, date: 20190309
Working on 2019: 124 of 152, date: 20190310
Working on 2019: 125 of 152, date: 20190311
Working on 2019: 126 of 152, date: 20190312
Working on 2019: 127 of 152, date: 20190313
Working on 2019: 128 of 152, date: 20190314
Working on 2019: 129 of 152, date: 20190315
Working on 2019: 130 of 152, date: 20190316
Working on 2019: 131 of 152, date: 20190317
Working on 2019: 132 of 152, date: 20190318
Working on 2019: 133 of 152, date: 20190319
Working on 2019: 134 of 152, date: 20190320
Working on 2019: 135 of 152, date: 20190321
Working on 2019: 136 of 152, date: 20190322
Working on 2019: 137 of 152, date: 20190323
Working on 2019: 138 of 152, date: 20190324
Working on 2019: 139 of 152, date: 20190325
Working on 2019: 140 of 152, date: 20190326
Working on 2019: 141 of 152, date: 20190327
Working on 2019: 142 of 152, date: 20190328
Working on 2019: 143 of 152, date: 20190329
Working on 2019: 144 of 152, dat

Working on 2021: 17 of 109, date: 20201211
Working on 2021: 18 of 109, date: 20201212
Working on 2021: 19 of 109, date: 20201213
Working on 2021: 20 of 109, date: 20201214
Working on 2021: 21 of 109, date: 20201215
Working on 2021: 22 of 109, date: 20201216
Working on 2021: 23 of 109, date: 20201217
Working on 2021: 24 of 109, date: 20201218
Working on 2021: 25 of 109, date: 20201219
Working on 2021: 26 of 109, date: 20201220
Working on 2021: 27 of 109, date: 20201221
Working on 2021: 28 of 109, date: 20201222
Working on 2021: 29 of 109, date: 20201223
Working on 2021: 30 of 109, date: 20201225
Working on 2021: 31 of 109, date: 20201226
Working on 2021: 32 of 109, date: 20201227
Working on 2021: 33 of 109, date: 20201228
Working on 2021: 34 of 109, date: 20201229
Working on 2021: 35 of 109, date: 20201230
Working on 2021: 36 of 109, date: 20201231
Working on 2021: 37 of 109, date: 20210101
Working on 2021: 38 of 109, date: 20210102
Working on 2021: 39 of 109, date: 20210103
Working on 